In [1]:
from llvm_ml.data import load_pyg_dataset
banned_ids = []
dataset = load_pyg_dataset("./data/ryzen3600_v11.cbuf", use_binary_opcode=False, banned_ids=banned_ids)
print(f"Training with {len(dataset)} samples")

Training with 44082 samples


In [3]:
from model.GraphBERT import ThroughputEstimator
from torch_geometric.loader import DataLoader
import torch
from lightning.pytorch.loggers import TensorBoardLogger
import lightning.pytorch as pl
from model import GraphBERT
from lightning.pytorch.callbacks import ModelSummary
from torch_geometric import compile

import importlib
importlib.reload(GraphBERT)

import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='TypedStorage is deprecated')

batch_size = 128
hidden_dim = 256
emb_size = 256

num_training = int(0.7 * len(dataset))
num_val = len(dataset) - num_training

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [num_training, num_val])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=6, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=6, drop_last=True)

model = ThroughputEstimator(21000, emb_size, batch_size, hidden_dim)
# model = compile(model)

logger = TensorBoardLogger("runs", name="bert")
logger.log_graph(model)
trainer = pl.Trainer(max_epochs=40, logger=logger, callbacks=[ModelSummary(max_depth=-1)])
trainer.fit(model, train_loader, val_loader)

Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                                    | Type                   | Params
------------------------------------------------------------------------------------
0  | embedding                               | MCEmbedding            | 5.4 M 
1  | embedding.embedding                     | Embedding              | 5.4 M 
2  | embedding.pos_encoding                  | PositionalEncoding     | 0     
3  | embedding.norm                          | LayerNorm              | 512   
4  | encoder                                 | MCEncoder              | 660 K 
5  | encoder.attention                       | MCAttention           

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

CompilationError: at 34:12:
def triton_(in_ptr0, in_ptr1, in_ptr2, in_ptr3, in_ptr4, in_ptr5, in_ptr6, out_ptr0, out_ptr1, out_ptr2, xnumel, rnumel, XBLOCK : tl.constexpr, RBLOCK : tl.constexpr):
    xnumel = 20040
    rnumel = 256
    xoffset = tl.program_id(0) * XBLOCK
    xindex = xoffset + tl.arange(0, XBLOCK)[:, None]
    xmask = xindex < xnumel
    rbase = tl.arange(0, RBLOCK)[None, :]
    x1 = (xindex // 4)
    tmp0 = tl.load(in_ptr0 + (x1), xmask)
    x0 = xindex % 4
    tmp2 = tl.load(in_ptr2 + (x1), xmask)
    _tmp5 = tl.zeros([XBLOCK, RBLOCK], tl.float32) + 0
    x3 = xindex
    tmp6 = tl.load(in_ptr4 + (x3), xmask)
    tmp7 = tl.load(in_ptr5 + (x1), xmask)
    for roffset in range(0, rnumel, RBLOCK):
        rindex = roffset + rbase
        rmask = rindex < rnumel
        r2 = rindex
        tmp1 = tl.load(in_ptr1 + (r2 + (256*x0) + (1024*tmp0)), rmask & xmask, eviction_policy='evict_last', other=0)
        tmp3 = tl.load(in_ptr3 + (r2 + (256*x0) + (1024*tmp2)), rmask & xma